### Acne Detection and Diagnosis

### Importing the Dataset

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

2024-08-01 03:13:04.188506: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 03:13:04.249196: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 03:13:04.330928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 03:13:04.413182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 03:13:04.428569: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 03:13:04.559074: I tensorflow/core/platform/cpu_feature_gu

In [2]:
#help(EarlyStopping)

### Data Augmentation
**Augment the training data to create more diverse training examples, improving model robustness**

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

### No augmentation for validation and test data; just rescale the pixel values

In [4]:
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

### Load and preprocess the training data

In [6]:
train_image_directory = "dataset/training/"


train_generator = train_datagen.flow_from_directory(
    directory=train_image_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary", # Binary Classification (acne or no acne)
)

Found 1646 images belonging to 2 classes.


### Load and preprocess the validation data

In [7]:
validation_image_directory = "dataset/validation/"


validation_generator = train_datagen.flow_from_directory(
    directory=validation_image_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary", # Binary Classification (acne or no acne)
)

Found 112 images belonging to 2 classes.


### Load and preprocess the test data

In [8]:
test_image_directory = "dataset/testing/"


test_generator = train_datagen.flow_from_directory(
    directory=validation_image_directory,
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary", # Binary Classification (acne or no acne)
)

Found 112 images belonging to 2 classes.


### Load the pre-trained VGG16 model, excluding  the top (fully connected) layers

In [10]:
base_model = VGG16(input_shape=(150, 150, 3), include_top=False, weights="imagenet")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 84s 1us/step


### Freeze the base model's layers to prevent them from being updated during training


In [11]:
base_model.trainable = False

In [13]:
print(base_model)

<Functional name=vgg16, built=True>


### Add custom layers on top of the base model

In [14]:
x = Flatten()(base_model.output)
x = Dense(512, activation = "relu")(x)
output = Dense(units=1, activation="sigmoid")(x)

In [17]:
x

<KerasTensor shape=(None, 512), dtype=float32, sparse=False, name=keras_tensor_23>

In [16]:
output

<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_24>

### Create the complete model

In [18]:
model = Model(inputs=base_model.input, outputs=output)

In [19]:
model

<Functional name=functional, built=True>

### Compile the model

In [20]:
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

### Use early stopping to prevent overfitting

In [22]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [23]:
early_stopping

### Training the model

In [24]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples //train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5,
    callbacks=[early_stopping]
)

Epoch 1/5


/home/agbor/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


51/51 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.6761 - loss: 0.8162 - val_accuracy: 0.8125 - val_loss: 0.3842
Epoch 2/5
 1/51 ━━━━━━━━━━━━━━━━━━━━ 1:53 2s/step - accuracy: 0.8125 - loss: 0.2898

2024-08-01 03:23:35.068623: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/agbor/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.8125 - loss: 0.2898 - val_accuracy: 0.8125 - val_loss: 0.3669
Epoch 3/5


2024-08-01 03:23:36.340155: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


51/51 ━━━━━━━━━━━━━━━━━━━━ 131s 3s/step - accuracy: 0.8814 - loss: 0.3138 - val_accuracy: 0.9271 - val_loss: 0.2361
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.7812 - loss: 0.5817 - val_accuracy: 0.6250 - val_loss: 1.0299
Epoch 5/5


2024-08-01 03:25:50.462930: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


51/51 ━━━━━━━━━━━━━━━━━━━━ 130s 3s/step - accuracy: 0.8905 - loss: 0.2645 - val_accuracy: 0.8438 - val_loss: 0.2933


In [27]:
### Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Accuracy: {test_acc:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.8698 - loss: 0.2907
Test Accuracy: 0.8646


### Making a single prediction

In [30]:
from keras.preprocessing import image
import numpy as np

# Load the image and resize it to 64x64
image_path = "dataset/testing/acne/acne-7_jpeg.rf.34ee37b930f592f787bb8d1bf6b37d30.jpg"

test_image = image.load_img(image_path, target_size=(150, 150))
test_image = image.img_to_array(test_image)

# Expand dimensions to add the batch size
test_image = np.expand_dims(test_image, axis=0)

# Predicting the result
result = model.predict(test_image)

train_generator.class_indices

if result[0][0] == 1:
    prediction = "Acne"
else:
    prediction = "Non Acne"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


In [31]:
print(prediction)

Non Acne
